### Dev - Debug token request


In [ ]:
from pathlib import Path
import sys
import os
import django
from datetime import date

# Add the parent directory to the system path and setup django
BASE_DIR = str(Path(os.getcwd()).resolve().parent.parent.parent.parent)

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

from tsosi.data.preparation.scipost.data import (
    get_scipost_raw_data,
    SCIPOST_TOKEN_URL,
)
from tsosi.app_settings import app_settings
from datetime import date
from pathlib import Path
import requests

today_str = date.today().strftime("%Y-%m-%d")
raw_data_file = f"{today_str}_scipost_raw.json"

raw_data_folder = Path.home() / "Nextcloud/TSOSI_data/scipost/0_raw"

raw_path = str(raw_data_folder / raw_data_file)
# get_scipost_raw_data(dest_file=raw_path)

print("Collecting OAuth2 token")
auth_data = app_settings.SCIPOST_AUTH

if auth_data is None:
    raise Exception("Missing TSOSI_SCIPOST_AUTH setting")

payload = {
    "grant_type": "password",
    "username": "TSOSI",
    "password": auth_data["password"],
    "scope": "read",
}
auth = (auth_data["client_id"], auth_data["client_secret"])
# print(auth)
# print(payload)
token_data: dict = requests.post(SCIPOST_TOKEN_URL, payload, auth=auth).json()

In [ ]:
token_data

In [ ]:
from urllib.parse import urlencode, quote_plus

qs = "&".join([f"{k}={v}" for k, v in payload.items()])
qs = urlencode(payload)

authentication = ":".join(auth)

print(
    f'curl -X POST -d "{qs}" -u "{authentication}" https://scipost.org/o/token/'
)
# token_data
quote_plus("contact@tsosi.org")

### 0 - Get SciPost data


In [ ]:
from pathlib import Path
import sys
import os
import django

# Add the parent directory to the system path and setup django
BASE_DIR = Path(os.getcwd()).resolve().parent.parent.parent.parent

if str(BASE_DIR) not in sys.path:
    sys.path.append(str(BASE_DIR))

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

from tsosi.data.preparation.scipost.data import get_scipost_raw_data
from pathlib import Path

raw_data_folder = BASE_DIR / "_no_git/fixtures/scipost/0_raw"
results = get_scipost_raw_data(dest_folder=raw_data_folder)

### 1 - Process collected data


In [ ]:
from pathlib import Path
import sys
import os
import django
import pandas as pd

# Add the parent directory to the system path and setup django
BASE_DIR = Path(os.getcwd()).resolve().parent.parent.parent.parent

if str(BASE_DIR) not in sys.path:
    sys.path.append(str(BASE_DIR))

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

date_str = "2025-06-24"
raw_data_folder = BASE_DIR / "_no_git/fixtures/scipost/0_raw"

subsidies = pd.read_json(
    str(raw_data_folder / f"{date_str}_scipost_subsidies.json")
)
payments = pd.read_json(
    str(raw_data_folder / f"{date_str}_scipost_payments.json")
)
collectives = pd.read_json(
    str(raw_data_folder / f"{date_str}_scipost_collectives.json")
)


from tsosi.data.preparation.scipost.data import (
    prepare_data,
)

res = prepare_data(payments, subsidies, collectives)

prepared_data_folder = BASE_DIR / "_no_git/fixtures/scipost/4_processed"
dest_file = str(prepared_data_folder / f"{date_str}_scipost_processed.json")
with open(dest_file, "w") as f:
    res.to_json(
        f,
        orient="records",
        indent=2,
    )

print(f"Prepared data dumped in {dest_file}")

In [ ]:
res

### 5 - Generate TSOSI data file


In [ ]:
from pathlib import Path
import sys
import os
import django
from datetime import date

# Add the parent directory to the system path and setup django
BASE_DIR = Path(os.getcwd()).resolve().parent.parent.parent.parent

if str(BASE_DIR) not in sys.path:
    sys.path.append(str(BASE_DIR))

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

from backend.tsosi.data.preparation.scipost.configs import get_file_config

date_data = date(2025, 6, 24)
date_str = date_data.strftime("%Y-%m-%d")
file_path = (
    BASE_DIR
    / f"_no_git/fixtures/scipost/4_processed/{date_str}_scipost_processed.json"
)
config = get_file_config(str(file_path), date_data)
config.generate_data_file()